In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
file_path1 = '/content/Crime-tweets-dataset.csv'

In [12]:
import pandas as pd

df = pd.read_csv(file_path1)
print("✅ Number of rows:", len(df))
print("✅ Number of columns:", len(df.columns))

✅ Number of rows: 400
✅ Number of columns: 8


In [13]:
df.info()           # shows row count, column names, non-null entries
df.head()           # shows first 5 rows
df['source'].value_counts()  # see how many from auto/manual

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  400 non-null    int64 
 1   Tweets      400 non-null    object
 2   id          400 non-null    int64 
 3   len         400 non-null    int64 
 4   date        400 non-null    object
 5   source      400 non-null    object
 6   likes       400 non-null    int64 
 7   retweets    400 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 25.1+ KB


,count
source,
Twitter Web App,210
Twitter for iPad,80
Hootsuite Inc.,54
Twitter for iPhone,29
Periscope,11
Twitter Media Studio,8
Instagram,5
Twitter for Android,3


In [2]:
# quick_start_labeling.py
import pandas as pd
from transformers import pipeline

def create_initial_dataset(tweet_file, sample_size=1000, output_file="labeled_dataset.csv"):
    """Create initial labeled dataset with minimal effort and save output"""

    # Load tweets
    df = pd.read_csv(tweet_file)

    if sample_size > len(df):
        sample_size = len(df)
        print(f"Warning: Requested sample size ({sample_size}) > dataset size ({len(df)}). Using full dataset.")

    df = df.sample(sample_size).copy()

    # Step 1: Zero-shot auto labeling
    classifier = pipeline("zero-shot-classification",
                         model="facebook/bart-large-mnli")

    categories = ["drug crime", "violent crime", "commercial crime",
                  "property crime", "traffic crime", "other crime", "non-crime"]

    high_confidence_data = []
    print("🔍 Auto-labeling high-confidence tweets...")

    for text in df['Tweets'].head(300):
        result = classifier(text, categories)
        if result['scores'][0] > 0.85:
            high_confidence_data.append({
                'text': text,
                'category': result['labels'][0],
                'confidence': result['scores'][0],
                'source': 'auto_high_conf'
            })

    auto_df = pd.DataFrame(high_confidence_data)
    print(f"✅ Auto labeled: {len(auto_df)}")

    # Step 2: Manual review
    remaining = df[~df['Tweets'].isin(auto_df['text'])].head(200)
    print("\n📝 Manual labeling required...")

    manual_data = []

    for text in remaining['Tweets']:
        print("\nText:", text)
        result = classifier(text, categories)

        for i in range(3):
            print(f"  {i}: {result['labels'][i]} ({result['scores'][i]:.3f})")

        choice = input("Enter index (0–2) or 's' to skip: ")
        if choice.isdigit() and 0 <= int(choice) <= 2:
            manual_data.append({
                'text': text,
                'category': result['labels'][int(choice)],
                'confidence': result['scores'][int(choice)],
                'source': 'manual_review'
            })

    manual_df = pd.DataFrame(manual_data)

    # ✅ Combine and Save
    final_df = pd.concat([auto_df, manual_df], ignore_index=True)
    final_df.to_csv(output_file, index=False)

    print(f"\n🎯 Saved final labeled dataset → {output_file}")
    print(f"📌 Total labeled: {len(final_df)}")

    return final_df

# Usage Example
labeled_data = create_initial_dataset("Crime-tweets-dataset.csv", output_file="crime_labeled_v1.csv")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


🔍 Auto-labeling high-confidence tweets...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Auto labeled: 95

📝 Manual labeling required...

Text: 🚨WANTED￼🚨for Multiple Burglaries the first occurred @ 206 East 124 St. #Manhattan @NYPD25pct on 3/10/22@ 7 A.M. The individual gains access from the basement of the building and removed property💰Reward up to $3500 Know who he is?￼📲Call 1-800-577-TIPS Calls are CONFIDENTIAL! https://t.co/fRLcB8iay7
  0: property crime (0.815)
  1: other crime (0.073)
  2: violent crime (0.050)
Enter index (0–2) or 's' to skip: 0

Text: NYC: Do you know where Isaiah Metz, AKA Decklyn McBride, is? This morning, inside a shelter at 257 West 30 St in Manhattan, he assaulted 2 Regional Fugitive Task Force Officers attempting to take a male into custody for outstanding warrants. Have info? Call @NYPDTips 800577TIPS https://t.co/OHV0oaK3CD
  0: violent crime (0.832)
  1: other crime (0.086)
  2: property crime (0.021)
Enter index (0–2) or 's' to skip: 0

Text: RT @NYPDPC: A morning subway ride on the “A” line with @NYPDFIRSTDEP Edward Caban, and @NYPDChi

In [4]:
file_path = '/content/crime_labeled_v1.csv'


In [5]:
import pandas as pd

df = pd.read_csv(file_path)
print("✅ Number of rows:", len(df))
print("✅ Number of columns:", len(df.columns))


✅ Number of rows: 291
✅ Number of columns: 4


In [6]:
df.info()           # shows row count, column names, non-null entries
df.head()           # shows first 5 rows
df['source'].value_counts()  # see how many from auto/manual


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        291 non-null    object 
 1   category    291 non-null    object 
 2   confidence  291 non-null    float64
 3   source      291 non-null    object 
dtypes: float64(1), object(3)
memory usage: 9.2+ KB


,count
source,
manual_review,196
auto_high_conf,95


In [14]:
import pandas as pd

# Load original tweets
original_df = pd.read_csv("/content/Crime-tweets-dataset.csv")

# Load your labeled output
labeled_df = pd.read_csv("/content/crime_labeled_v1.csv")

print("Original:", len(original_df))
print("Labeled:", len(labeled_df))


Original: 400
Labeled: 291


In [15]:
labeled_texts = set(labeled_df['text'])
original_texts = set(original_df['Tweets'])

# Find the difference
unlabeled_texts = original_texts - labeled_texts
print("Unlabeled tweets:", len(unlabeled_texts))


Unlabeled tweets: 109


In [16]:
unlabeled_df = original_df[~original_df['Tweets'].isin(labeled_df['text'])]
print("✅ Unlabeled count:", len(unlabeled_df))
unlabeled_df.head()


✅ Unlabeled count: 109


,Unnamed: 0,Tweets,id,len,date,source,likes,retweets
6,6,RT @NYPDFIRSTDEP: From @NYPDSchools Safety Age...,1518258674904584193,140,2022-04-24 16:00:50+00:00,Twitter Web App,0,23
7,7,"Here is a closer look at the suspect. Again, i...",1518228462418415617,145,2022-04-24 14:00:47+00:00,Twitter Web App,24,32
8,8,🚨WANTED for GLA/Leaving The Scene of an Accide...,1518228457959866369,307,2022-04-24 14:00:46+00:00,Twitter Web App,138,107
19,19,🚨WANTED for a FORCIBLE TOUCHING: Do you know h...,1517503726331772928,277,2022-04-22 14:00:56+00:00,Hootsuite Inc.,45,55
20,20,Cash for Guns with @QueensDAKatz \n \nNO QUEST...,1517480870680088579,274,2022-04-22 12:30:07+00:00,Hootsuite Inc.,39,16


In [17]:
unlabeled_df.to_csv("/content/unlabeled_remaining.csv", index=False)
print("💾 Saved unlabeled tweets to unlabeled_remaining.csv")


💾 Saved unlabeled tweets to unlabeled_remaining.csv


In [18]:
labeled_ratio = len(labeled_df) / len(original_df) * 100
print(f"✅ {labeled_ratio:.1f}% tweets labeled, {100 - labeled_ratio:.1f}% skipped/unlabeled.")


✅ 72.8% tweets labeled, 27.2% skipped/unlabeled.


In [19]:
# quick_start_labeling.py
import pandas as pd
from transformers import pipeline

def create_initial_dataset(tweet_file, sample_size=1000, output_file="labeled_dataset.csv"):
    """Create initial labeled dataset with minimal effort and save output"""

    # Load tweets
    df = pd.read_csv(tweet_file)

    if sample_size > len(df):
        sample_size = len(df)
        print(f"Warning: Requested sample size ({sample_size}) > dataset size ({len(df)}). Using full dataset.")

    df = df.sample(sample_size).copy()

    # Step 1: Zero-shot auto labeling
    classifier = pipeline("zero-shot-classification",
                         model="facebook/bart-large-mnli")

    categories = ["drug crime", "violent crime", "commercial crime",
                  "property crime", "traffic crime", "other crime", "non-crime"]

    high_confidence_data = []
    print("🔍 Auto-labeling high-confidence tweets...")

    for text in df['Tweets'].head(300):
        result = classifier(text, categories)
        if result['scores'][0] > 0.85:
            high_confidence_data.append({
                'text': text,
                'category': result['labels'][0],
                'confidence': result['scores'][0],
                'source': 'auto_high_conf'
            })

    auto_df = pd.DataFrame(high_confidence_data)
    print(f"✅ Auto labeled: {len(auto_df)}")

    # Step 2: Manual review
    remaining = df[~df['Tweets'].isin(auto_df['text'])].head(200)
    print("\n📝 Manual labeling required...")

    manual_data = []

    for text in remaining['Tweets']:
        print("\nText:", text)
        result = classifier(text, categories)

        for i in range(3):
            print(f"  {i}: {result['labels'][i]} ({result['scores'][i]:.3f})")

        choice = input("Enter index (0–2) or 's' to skip: ")
        if choice.isdigit() and 0 <= int(choice) <= 2:
            manual_data.append({
                'text': text,
                'category': result['labels'][int(choice)],
                'confidence': result['scores'][int(choice)],
                'source': 'manual_review'
            })

    manual_df = pd.DataFrame(manual_data)

    # ✅ Combine and Save
    final_df = pd.concat([auto_df, manual_df], ignore_index=True)
    final_df.to_csv(output_file, index=False)

    print(f"\n🎯 Saved final labeled dataset → {output_file}")
    print(f"📌 Total labeled: {len(final_df)}")

    return final_df

# Usage Example
labeled_data = create_initial_dataset("unlabeled_remaining.csv", output_file="crime_labeled_v2.csv")


Device set to use cuda:0


🔍 Auto-labeling high-confidence tweets...
✅ Auto labeled: 43

📝 Manual labeling required...

Text: 🚨WANTED🚨for A ROBBERY @ Metro PCS)located@ 5585 Broadway #Bronx @NYPD50pct on 3/25/22@ 6:17P.M. The individual displayed a firearm and demanded cash.  The individual removed $1,764 cash &amp; fled.💰Reward up to $3500 Know who he is?￼📲Call 1-800-577-TIPS Calls are CONFIDENTIAL! https://t.co/hVxz4WkyKW
  0: violent crime (0.658)
  1: commercial crime (0.170)
  2: other crime (0.095)
Enter index (0–2) or 's' to skip: 0

Text: RT @NYPD104Pct: Don’t count your 🐥🐓 before they hatch! Lieutenant Hellmer and your 104 NCO’s held a joint operation with @ASPCA and @NYCACC…
  0: other crime (0.364)
  1: non-crime (0.234)
  2: violent crime (0.146)
Enter index (0–2) or 's' to skip: 0

Text: #GoodMorning NYC!

Your NYPD officers are always standing at the ready, 24/7 to protect and serve all New Yorkers. https://t.co/Q3JCErtEP3
  0: non-crime (0.261)
  1: other crime (0.213)
  2: commercial crime (0.141

In [20]:
import pandas as pd

# Load datasets
df1 = pd.read_csv('crime_labeled_v1.csv')
df2 = pd.read_csv('crime_labeled_v2.csv')

# Merge row-wise
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save to a new CSV
merged_df.to_csv('final_crime_tweets_dataset.csv', index=False)


In [21]:
# Print shape of the merged DataFrame
print(merged_df.shape)


(395, 4)


In [22]:
num_rows = merged_df.shape[0]      # 395
num_columns = merged_df.shape[1]   # depends on your dataset
print("Rows:", num_rows)
print("Columns:", num_columns)


Rows: 395
Columns: 4


In [23]:
merged_df.info()           # shows row count, column names, non-null entries
merged_df.head()           # shows first 5 rows
merged_df['source'].value_counts()  # see how many from auto/manual

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        395 non-null    object 
 1   category    395 non-null    object 
 2   confidence  395 non-null    float64
 3   source      395 non-null    object 
dtypes: float64(1), object(3)
memory usage: 12.5+ KB


,count
source,
manual_review,257
auto_high_conf,138
